## Cargar bibliotecas

In [2]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.5 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
#from deep_translator import GoogleTranslator
import time


# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
#from nltk.stem import WordNetLemmatizer

#Replace all non alphabets
from unidecode import unidecode

#For lematization
import spacy

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

In [4]:
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = stopwords.words("spanish")
stemmer = SnowballStemmer("spanish")#extrae la raíz de una palabra
!spacy download es
lematiza = spacy.load('es_core_news_sm')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


2022-08-23 15:07:53.233555: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 455 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


## Tratamiento stopwords

In [5]:
stop_words.extend(['ye','para','irrtificar','santiaguir','e','xhlz','zon','qie','sexy','bta','zuleerma','? ar','saimecripto',
                  'eliar aguilar','gn','mercantilbancouniversal','bancomercantil'])
#set(stop_words) # volver a usar stopwords original


In [6]:
elimina = ['no','estado','sin','como']
stop_words = [x for x in stop_words if x  not in elimina]

## Carga herramientas

In [11]:
emojis = pd.read_excel('/content/drive/MyDrive/Mercantil/dic_emoji.xlsx')
emojis.set_index('emoji',inplace=True)
emojis = emojis.to_dict()
emojis = emojis['equivale']
print(emojis)

{'😈': 'diablo', '😀': 'sonrisa', '😍': 'encantar', '😄': 'sonrisa', '😂': 'risa', '😉': 'guiño', '😊': 'sonrisa', '🤣': 'risa', '😘': 'beso', '🤡': 'payaso', '👻': 'fantasma', '😆': 'burla', '🥰': 'encantar', '😻': 'encantar', '😋': 'feliz', '😒': 'desagradar', '😓': 'desagradar', '😔': 'triste', '😖': 'triste', '😚': 'beso', '🏦': 'banco', '😜': 'burla', '😝': 'burla', '😞': 'triste', '😠': 'enojo', '😡': 'enojo', '😢': 'triste', '😣': 'frustracion', '😤': 'enojo', '😥': 'preocupacion', '😩': 'frustracion', '😪': 'triste', '😭': 'triste', '😫': 'frustracion', '😰': 'preocupacion', '😳': 'sonrojado', '😸': 'risa', '😹': 'risa', '😺': 'risa', '😿': 'triste', '🙅': 'bloqueo', '🙏': 'rezar', '❤': 'corazon', '👍': 'bien', '👎': 'mal', '👏': 'aplauso', '👪': 'familia', '👿': 'enojado', '💩': 'mierda', '💪': 'fuerza', '😐': 'neutral', '😑': 'serio', '😕': 'confundido', '😟': 'preocupacion', '😧': 'preocupacion', '🌞': 'sol', '🇻🇪': 'venezuela', '😨': 'miedo', '😱': 'miedo', ':)': 'sonrisa', '💜': 'corazon', '💝': 'regalo', '🚗': 'carro', '🚕': 'taxi',

In [10]:
with open('/content/drive/MyDrive/Mercantil/extencion_lematizador') as f:
    palabras = dict(i.rstrip().split(None, 1) for i in f)
print(palabras)

{'madurar': 'maduro', 'clavar': 'clave', 'paginar': 'pagina', 'sera': 'ser', 'seguira': 'seguir', 'divisar': 'divisa', 'lonuevo': 'nuevo', 'deberiar': 'deber', 'cumpliraa+-os': 'cumpleanos', 'nadar': 'nada', 'pudrir': 'poder', 'sii': 'si', 'ye': 'si', 'parir': 'para', 'cariaa+-o': 'carino', 'app': 'aplicacion', 'gracia': 'gracias', 'fallir': 'fallar', 'aa+-os': 'año', 'huma?anaird': 'humanidad', 'linear': 'linea', 'contactarer': 'contactar', 'naciinal': 'nacional', 'ma?anada': 'mañana', 'estarfaeued': 'estar', 'eaa': 'esa', 'quebhay': 'que hay', 'enviir': 'enviar', 'mercanril': 'mercantil', 'estrellar': 'estrella', 'malisimir': 'malo', 'gustariar': 'gustar', 'cencilla': 'sencillo', 'sencillar': 'sencillo', 'agilizarizen': 'agilizar', 'seguo': 'seguro', 'graciasscontar': 'gracias', 'line': 'online', 'objetivar': 'objetivo', 'personar': 'personal', 'colocars': 'colocar', 'presi': 'presidente', 'gnar': 'ganar', 'dejarra': 'dejar', 'aa+-o': 'año', 'nformatico': 'informatico', 'pagin': 'pag

In [ ]:
def preprocess(tweet, stem=False):
    
    # Create Lemmatizer and Stemmer.
    #wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    tweet = tweet.lower()
        
    # Replace all URls with 'URL'
    tweet = re.sub(urlPattern,' URL',tweet)
    # Replace all emojis.
    for emoji in emojis.keys():
      tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji]+" ")        
    # Replace @USERNAME to 'USER'.
    tweet = re.sub(userPattern,'', tweet)        
    # Replace all non alphabets.
    tweet = unidecode(tweet)
    # Replace 3 or more consecutive letters by 2 letter.
    tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
    #Lematiza
    tweet = lematiza(tweet)
    lemmas = [tok.lemma_.lower() for tok in tweet]
    tokens = []
    for token in lemmas:
          # Checking if the word is a stopword.
          if token not in stop_words:
            #cheching if word has more than one letter
            for palabra in palabras.keys():
              token = token.replace(palabra, palabras[palabra])
            if len(token)>1:
              if stem:
                tokens.append(stemmer.stem(token))
              else:
                tokens.append(token)
    return " ".join(tokens)


        

## Carga y revisa data

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Mercantil/banco_train_V2.xlsx')

In [ ]:
df.groupby('target').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,date,user,text
target,,,,,,,
NEGATIVE,3046,3046,3046,3046,3046,3046,3046
POSITIVE,3046,3046,3046,3046,3046,3046,3046


In [ ]:
df.text = df.text.apply(lambda x: str(x))
df.text = df.text.apply(lambda x: preprocess(x))

In [ ]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,target,date,user,text
0,0.0,0.0,0.0,0.0,NEGATIVE,2020-02-05 17:20:40,Katiuska Tenorio,desinstalar pedido propio aplicacion ahora no ...
1,1.0,1.0,1.0,1.0,NEGATIVE,2020-05-30 20:34:26,Maria Estrada,super mala actualizacion verdad vez ayudar hhh...
2,2.0,2.0,2.0,2.0,NEGATIVE,2021-01-28 11:18:30,Omar Marquez,no haber poder ingresar pedir codigo aceptacio...
3,3.0,3.0,3.0,3.0,NEGATIVE,2021-06-01 14:27:15,Evaristo Bartolo Rivero,estrella instalar no haber poder abrir arrojar...
4,4.0,4.0,4.0,4.0,NEGATIVE,2019-05-23 22:12:19,johan palacio,volver caga .. tener rato intentar hhhacer pag...
...,...,...,...,...,...,...,...,...
6087,6087.0,6087.0,6087.0,6087.0,NEGATIVE,2022-08-07 00:00:00,Admin,deficiente atencion
6088,6088.0,6088.0,6088.0,6088.0,NEGATIVE,2022-08-07 00:00:00,Admin,pobre atencion oficinaaa
6089,6089.0,6089.0,6089.0,6089.0,POSITIVE,2022-08-07 00:00:00,Admin,mejor atencion mercado
6090,6090.0,6090.0,6090.0,6090.0,POSITIVE,2022-08-07 00:00:00,Admin,buen atencion gracias


In [ ]:
df.to_excel('/content/drive/MyDrive/Mercantil/banco_train_V3.xlsx')